In [1]:
import numpy as np
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn import datasets, metrics, cross_validation

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [62]:
y_train

array([ 18.5,  19.6,  33.2,  13.1,   7.5,  13.6,  17.4,   8.4,  35.4,
        24. ,  13.4,  26.2,   7.2,  13.1,  24.5,  37.2,  25. ,  24.1,
        16.6,  32.9,  36.2,  11. ,   7.2,  22.8,  28.7,  14.4,  24.4,
        18.1,  22.5,  20.5,  15.2,  17.4,  13.6,   8.7,  18.2,  35.4,
        31.7,  33. ,  22.2,  20.4,  23.9,  25. ,  12.7,  29.1,  12. ,
        17.7,  27. ,  20.6,  10.2,  17.5,  19.7,  29.8,  20.5,  14.9,
        10.9,  19.5,  22.7,  19.5,  24.6,  25. ,  24.5,  50. ,  14.3,
        11.8,  31. ,  28.7,  16.2,  43.5,  25. ,  22. ,  19.9,  22.1,
        46. ,  22.9,  20.2,  43.1,  34.6,  13.8,  24.3,  21.5,  24.4,
        21.2,  23.8,  26.6,  25.1,   9.6,  19.4,  19.4,   9.5,  14. ,
        26.5,  13.8,  34.7,  16.3,  21.7,  17.5,  15.6,  20.9,  21.7,
        12.7,  18.5,  23.7,  19.3,  12.7,  21.6,  23.2,  29.6,  21.2,
        23.8,  17.1,  22. ,  36.5,  18.8,  21.9,  23.1,  20.2,  17.4,
        37. ,  24.1,  36.2,  15.7,  32.2,  13.5,  17.9,  13.3,  11.7,
        41.7,  18.4,

In [61]:
sample = cross_validation.StratifiedKFold(
        train_y, # Samples to split in K folds
        n_folds=5, # Number of folds. Must be at least 2.
        shuffle=True, # Whether to shuffle each stratification of the data before splitting into batches.
        random_state=30 # pseudo-random number generator state used for shuffling
    )

for k, l in sample:
    print(len(train_x[k]))

305
295
298
311
307


/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


In [47]:
def objective(params):
    # https://github.com/dmlc/xgboost/issues/1034
    params['max_depth'] = int(params['max_depth'])

    skf = cross_validation.StratifiedKFold(
        train_y, # Samples to split in K folds
        n_folds=5, # Number of folds. Must be at least 2.
        shuffle=True, # Whether to shuffle each stratification of the data before splitting into batches.
        random_state=30 # pseudo-random number generator state used for shuffling
    )

    boost_rounds = []
    score = []

    for train, test in skf:
        _train_x, _test_x, _train_y, _test_y = \
            train_x[train], train_x[test], train_y[train], train_y[test]

        train_xd = xgb.DMatrix(_train_x, label=_train_y)
        test_xd = xgb.DMatrix(_test_x, label=_test_y)
        watchlist = [(train_xd, 'train'),(test_xd, 'eval')]

        model = xgb.train(
            params,
            train_xd,
            num_boost_round=100,
            evals=watchlist,
            early_stopping_rounds=30
        )

        boost_rounds.append(model.best_iteration)
        score.append(model.best_score)

    print('average of best iteration:', np.average(boost_rounds))
    return {'loss': np.average(score), 'status': STATUS_OK}

def optimize(trials):
    space = {
         'learning_rate': hp.quniform('learning_rate', 0.1, 1, 0.05),
         'objective': 'reg:linear',
         'booster': hp.choice('booster', ['dart']),
         'sample_type': 'uniform',
         'normalize_type': 'tree',
         'rate_drop': hp.quniform('rate_drop', 0.1, 0.8, 0.1),
         'skip_drop': hp.quniform('skip_drop', 0.1, 0.8, 0.1),
         'max_depth' : hp.quniform('max_depth', 6, 13, 1),
         'silent': True,
    }

    # minimize the objective over the space
    best_params = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        trials=trials,
        max_evals=50
    )

    return best_params


In [48]:
boston = datasets.load_boston()

In [49]:
X = boston.data
y = boston.target
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, random_state=0)


In [50]:
trials = Trials()
best_params = optimize(trials)
print(best_params)
print(objective(best_params))

[0]	train-rmse:7.68231	eval-rmse:8.10395
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 30 rounds.
[1]	train-rmse:14.3735	eval-rmse:6.32291
[2]	train-rmse:5.32524	eval-rmse:17.9314
[3]	train-rmse:5.08396	eval-rmse:5.54134
[4]	train-rmse:4.30703	eval-rmse:15.5887
[5]	train-rmse:4.29693	eval-rmse:20.8706
[6]	train-rmse:21.2589	eval-rmse:4.87456
[7]	train-rmse:2.32534	eval-rmse:4.33959
[8]	train-rmse:18.9063	eval-rmse:17.4063
[9]	train-rmse:1.40563	eval-rmse:3.97942
[10]	train-rmse:18.7725	eval-rmse:17.0004
[11]	train-rmse:0.969249	eval-rmse:16.972
[12]	train-rmse:18.4724	eval-rmse:18.2899
[13]	train-rmse:0.737809	eval-rmse:16.3794
[14]	train-rmse:0.427103	eval-rmse:3.83706
[15]	train-rmse:0.323314	eval-rmse:19.1709
[16]	train-rmse:0.236635	eval-rmse:17.2176
[17]	train-rmse:0.307626	eval-rmse:20.0138
[18]	train-rmse:0.2269	eval-rmse:22.3137
[19]	train-rmse:0.305249	eval-rmse:21.2223
[20]	train-rmse:21.933

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


[76]	train-rmse:0.074851	eval-rmse:3.53234
[77]	train-rmse:17.5277	eval-rmse:3.53952
[78]	train-rmse:17.6688	eval-rmse:3.53863
[79]	train-rmse:16.7606	eval-rmse:17.4775
[80]	train-rmse:0.034822	eval-rmse:21.6487
[81]	train-rmse:0.062775	eval-rmse:3.52485
[82]	train-rmse:17.7293	eval-rmse:3.52938
[83]	train-rmse:18.1005	eval-rmse:3.53404
[84]	train-rmse:19.247	eval-rmse:3.53746
[85]	train-rmse:0.052174	eval-rmse:23.4789
[86]	train-rmse:0.030015	eval-rmse:19.493
[87]	train-rmse:19.5347	eval-rmse:3.53743
[88]	train-rmse:17.1714	eval-rmse:3.54017
[89]	train-rmse:0.046434	eval-rmse:18.2659
[90]	train-rmse:0.027243	eval-rmse:3.53634
[91]	train-rmse:0.017208	eval-rmse:21.1262
[92]	train-rmse:0.010498	eval-rmse:3.53866
[93]	train-rmse:18.1304	eval-rmse:3.54002
[94]	train-rmse:0.038405	eval-rmse:3.52818
[95]	train-rmse:16.7938	eval-rmse:20.2012
[96]	train-rmse:0.033606	eval-rmse:18.6337
[97]	train-rmse:20.4181	eval-rmse:3.53061
[98]	train-rmse:17.2098	eval-rmse:3.52155
[99]	train-rmse:0.048001	

[92]	train-rmse:0.011432	eval-rmse:3.68037
[93]	train-rmse:18.2257	eval-rmse:3.68097
[94]	train-rmse:0.035024	eval-rmse:3.68331
[95]	train-rmse:16.8656	eval-rmse:19.5623
[96]	train-rmse:0.028181	eval-rmse:18.1186
[97]	train-rmse:20.5767	eval-rmse:3.68556
[98]	train-rmse:17.2875	eval-rmse:3.68905
[99]	train-rmse:0.04487	eval-rmse:3.68239
[0]	train-rmse:7.4547	eval-rmse:7.56497
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 30 rounds.
[1]	train-rmse:14.3722	eval-rmse:5.91474
[2]	train-rmse:4.85349	eval-rmse:17.3488
[3]	train-rmse:4.56747	eval-rmse:5.26799
[4]	train-rmse:3.9825	eval-rmse:14.9913
[5]	train-rmse:3.9308	eval-rmse:20.3118
[6]	train-rmse:21.3478	eval-rmse:4.74114
[7]	train-rmse:2.01467	eval-rmse:3.78146
[8]	train-rmse:18.9791	eval-rmse:16.8932
[9]	train-rmse:1.3512	eval-rmse:3.62557
[10]	train-rmse:18.8443	eval-rmse:16.5362
[11]	train-rmse:0.917481	eval-rmse:16.3856
[12]	train-rmse:18.5414	eva

[1]	train-rmse:4.51717	eval-rmse:4.88107
[2]	train-rmse:1.79625	eval-rmse:4.49237
[3]	train-rmse:1.01365	eval-rmse:23.676
[4]	train-rmse:0.766679	eval-rmse:4.55136
[5]	train-rmse:0.531562	eval-rmse:4.63834
[6]	train-rmse:11.7728	eval-rmse:4.23741
[7]	train-rmse:1.10818	eval-rmse:4.17371
[8]	train-rmse:1.31212	eval-rmse:3.93445
[9]	train-rmse:0.656951	eval-rmse:4.02178
[10]	train-rmse:0.772969	eval-rmse:3.98276
[11]	train-rmse:0.477424	eval-rmse:3.92422
[12]	train-rmse:0.260614	eval-rmse:3.92901
[13]	train-rmse:12.6204	eval-rmse:3.93472
[14]	train-rmse:0.213238	eval-rmse:3.92903
[15]	train-rmse:0.143051	eval-rmse:3.93203
[16]	train-rmse:0.082918	eval-rmse:3.9277
[17]	train-rmse:9.50165	eval-rmse:11.7974
[18]	train-rmse:0.02841	eval-rmse:3.92774
[19]	train-rmse:0.017701	eval-rmse:3.92868
[20]	train-rmse:0.009249	eval-rmse:3.92764
[21]	train-rmse:16.1178	eval-rmse:12.1529
[22]	train-rmse:22.29	eval-rmse:12.7969
[23]	train-rmse:0.001367	eval-rmse:3.92726
[24]	train-rmse:0.000938	eval-rmse:

...

[42]	train-rmse:0.231829	eval-rmse:3.38675
[43]	train-rmse:0.143746	eval-rmse:3.77286
[44]	train-rmse:0.091069	eval-rmse:3.37092
[45]	train-rmse:0.130516	eval-rmse:3.37043
[46]	train-rmse:0.082788	eval-rmse:4.70088
[47]	train-rmse:3.17005	eval-rmse:3.67186
[48]	train-rmse:0.104224	eval-rmse:4.07434
[49]	train-rmse:2.7948	eval-rmse:5.67684
[50]	train-rmse:0.181241	eval-rmse:3.34066
[51]	train-rmse:2.23175	eval-rmse:8.14756
[52]	train-rmse:0.28561	eval-rmse:3.32835
[53]	train-rmse:0.372578	eval-rmse:3.32402
[54]	train-rmse:5.67886	eval-rmse:3.52135
[55]	train-rmse:3.27335	eval-rmse:11.5712
[56]	train-rmse:0.469222	eval-rmse:3.32469
[57]	train-rmse:0.232069	eval-rmse:5.59221
[58]	train-rmse:0.260236	eval-rmse:3.9319
[59]	train-rmse:6.91084	eval-rmse:7.70638
[60]	train-rmse:0.299492	eval-rmse:3.27701
[61]	train-rmse:6.77803	eval-rmse:3.26342
[62]	train-rmse:3.31466	eval-rmse:4.38674
[63]	train-rmse:0.124598	eval-rmse:3.19802
[64]	train-rmse:3.69881	eval-rmse:5.98481
[65]	train-rmse:5.31662

[30]	train-rmse:0.680827	eval-rmse:17.5063
[31]	train-rmse:0.604772	eval-rmse:3.28666
[32]	train-rmse:0.63815	eval-rmse:15.8716
[33]	train-rmse:16.4583	eval-rmse:16.7267
[34]	train-rmse:21.2215	eval-rmse:17.9948
[35]	train-rmse:0.549478	eval-rmse:20.0667
[36]	train-rmse:0.568368	eval-rmse:19.193
[37]	train-rmse:0.51831	eval-rmse:20.3965
[38]	train-rmse:14.8922	eval-rmse:3.23062
[39]	train-rmse:17.8473	eval-rmse:3.22311
[40]	train-rmse:23.5425	eval-rmse:3.22889
[41]	train-rmse:20.7796	eval-rmse:14.1843
[42]	train-rmse:20.9933	eval-rmse:16.659
[43]	train-rmse:20.5955	eval-rmse:19.9613
[44]	train-rmse:17.9993	eval-rmse:3.21778
[45]	train-rmse:19.8554	eval-rmse:20.3216
[46]	train-rmse:14.9176	eval-rmse:3.21993
[47]	train-rmse:16.2565	eval-rmse:17.6806
[48]	train-rmse:0.36575	eval-rmse:19.4888
[49]	train-rmse:0.389555	eval-rmse:20.0626
[50]	train-rmse:0.409181	eval-rmse:3.22553
[51]	train-rmse:0.428277	eval-rmse:17.5029
[52]	train-rmse:0.45906	eval-rmse:19.6914
[53]	train-rmse:0.482446	eval

[18]	train-rmse:0.758639	eval-rmse:21.1779
[19]	train-rmse:0.795392	eval-rmse:19.8389
[20]	train-rmse:22.6154	eval-rmse:4.43857
[21]	train-rmse:0.879678	eval-rmse:18.2927
[22]	train-rmse:0.794829	eval-rmse:17.0846
[23]	train-rmse:18.2901	eval-rmse:22.8396
[24]	train-rmse:20.6087	eval-rmse:20.2523
[25]	train-rmse:23.8152	eval-rmse:18.8768
[26]	train-rmse:22.4058	eval-rmse:16.5604
[27]	train-rmse:0.745197	eval-rmse:16.9053
[28]	train-rmse:21.9546	eval-rmse:4.38163
[29]	train-rmse:0.521225	eval-rmse:4.40313
[30]	train-rmse:0.550832	eval-rmse:16.9643
[31]	train-rmse:0.510767	eval-rmse:4.38995
[32]	train-rmse:0.534787	eval-rmse:14.8467
[33]	train-rmse:16.5517	eval-rmse:15.7678
[34]	train-rmse:21.3705	eval-rmse:17.5042
[35]	train-rmse:0.436401	eval-rmse:19.1251
[36]	train-rmse:0.457616	eval-rmse:18.7812
[37]	train-rmse:0.41489	eval-rmse:19.5148
[38]	train-rmse:14.9104	eval-rmse:4.36902
[39]	train-rmse:17.9149	eval-rmse:4.36497
[40]	train-rmse:23.6873	eval-rmse:4.3563
[41]	train-rmse:20.903	e

[28]	train-rmse:22.0883	eval-rmse:3.06886
[29]	train-rmse:0.521689	eval-rmse:3.08671
[30]	train-rmse:0.551696	eval-rmse:16.2106
[31]	train-rmse:0.491912	eval-rmse:3.0654
[32]	train-rmse:0.524002	eval-rmse:14.0781
[33]	train-rmse:16.6427	eval-rmse:15.1427
[34]	train-rmse:21.4844	eval-rmse:16.816
[35]	train-rmse:0.448411	eval-rmse:18.2385
[36]	train-rmse:0.470251	eval-rmse:18.0327
[37]	train-rmse:0.385733	eval-rmse:18.7626
[38]	train-rmse:15.0358	eval-rmse:3.01149
[39]	train-rmse:18.0438	eval-rmse:3.00448
[40]	train-rmse:23.8445	eval-rmse:3.00816
[41]	train-rmse:20.9973	eval-rmse:13.2126
[42]	train-rmse:21.2816	eval-rmse:15.5044
[43]	train-rmse:20.8771	eval-rmse:18.7316
[44]	train-rmse:18.2329	eval-rmse:3.00608
[45]	train-rmse:20.085	eval-rmse:19.1627
[46]	train-rmse:15.1056	eval-rmse:3.0044
[47]	train-rmse:16.4178	eval-rmse:16.387
[48]	train-rmse:0.337098	eval-rmse:17.8364
[49]	train-rmse:0.357825	eval-rmse:18.6354
[50]	train-rmse:0.375839	eval-rmse:2.99064
[51]	train-rmse:0.397362	eval

XGBoostError: b'[16:04:23] src/gbm/gbm.cc:20: Unknown gbm type 0\n\nStack trace returned 7 entries:\n[bt] (0) 0   libxgboost.dylib                    0x00000001180e8c88 _ZN4dmlc15LogMessageFatalD2Ev + 40\n[bt] (1) 1   libxgboost.dylib                    0x00000001180e6df9 _ZN4dmlc15LogMessageFatalD1Ev + 9\n[bt] (2) 2   libxgboost.dylib                    0x000000011812bda9 _ZN7xgboost15GradientBooster6CreateERKNSt3__112basic_stringIcNS1_11char_traitsIcEENS1_9allocatorIcEEEERKNS1_6vectorINS1_10shared_ptrINS_7DMatrixEEENS5_ISD_EEEEf + 569\n[bt] (3) 3   libxgboost.dylib                    0x00000001180ec426 _ZN7xgboost11LearnerImpl13LazyInitModelEv + 758\n[bt] (4) 4   libxgboost.dylib                    0x00000001180fc2c0 XGBoosterUpdateOneIter + 80\n[bt] (5) 5   _ctypes.cpython-36m-darwin.so       0x000000010d6632c7 ffi_call_unix64 + 79\n[bt] (6) 6   ???                                 0x00007fff540bdd40 0x0 + 140734603451712\n'

In [42]:
train_xd = xgb.DMatrix(np.array(train_x), label=train_y)
bst = xgb.train(best_params, train_xd, num_boost_round=100)

pred_y = bst.predict(xgb.DMatrix(test_x))
mse = metrics.mean_squared_error(test_y, pred_y)
print(mse)

18.5629755054


In [51]:
best_params

{'booster': 0,
 'learning_rate': 0.25,
 'max_depth': 7,
 'rate_drop': 0.30000000000000004,
 'skip_drop': 0.6000000000000001}

In [44]:
best_params_add = best_params.copy()
best_params_add.update({'booster':'dart',
                        'objective': 'reg:linear',
                        'sample_type': 'uniform',
                       'normalize_type': 'tree'})
best_params_add

{'booster': 'dart',
 'learning_rate': 0.15000000000000002,
 'max_depth': 11,
 'normalize_type': 'tree',
 'objective': 'reg:linear',
 'rate_drop': 0.1,
 'sample_type': 'uniform',
 'skip_drop': 0.5}

In [45]:
bst = xgb.train(best_params_add, train_xd, num_boost_round=100)

In [46]:
pred_y = bst.predict(xgb.DMatrix(test_x))
mse = metrics.mean_squared_error(test_y, pred_y)
print(mse)

16.2019610281
